<a href="https://colab.research.google.com/github/Blacksujit/Deep-Learning-Specialization-Repo/blob/main/DoctorChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio faiss-cpu PyPDF2 openai==0.28 tiktoken -U langchain-community

In [ ]:
import gradio as gr
import openai
import faiss
import numpy as np
import requests
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
import json


In [ ]:
# OpenAI ve diğer API anahtarlarını ayarlayın
openai_api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # OpenAI API anahtarınızı buraya ekleyin
openai.api_key = openai_api_key

weather_api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # OpenWeatherMap API anahtarınızı buraya ekleyin
exchange_api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # Exchangeratesapi.io API anahtarınızı buraya ekleyin


In [ ]:
# PDF dosyalarının yolları
pdf_paths = ['/content/Current Essentials of Medicine.pdf'
]

# FAISS indeksi ve belgeler için global değişkenler
vector_index = None
documents = []
chat_history = []

In [ ]:
# PDF'leri okuma ve indeksleme fonksiyonu
def index_pdfs():
    global vector_index, documents

    for pdf_path in pdf_paths:
        pdf_reader = PdfReader(pdf_path)
        text = ""
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        documents.append(text)

    combined_text = " ".join(documents)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_text(combined_text)

    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    vectors = embeddings.embed_documents(texts)

    vector_array = np.array(vectors)

    index = faiss.IndexFlatL2(vector_array.shape[1])
    index.add(vector_array)

    vector_index = index

    print("Bilgi tabanı başarıyla oluşturuldu!")

index_pdfs()

<ipython-input-8-b3abdacc56b4>:19: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


Bilgi tabanı başarıyla oluşturuldu!


In [ ]:
# Hava durumu verilerini çekmek için fonksiyon
def fetch_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric&lang=tr"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        temp = data['main']['temp']
        weather_description = data['weather'][0]['description']
        return f"{location} için anlık hava durumu: {temp}°C ve {weather_description}."
    else:
        return "Hava durumu bilgilerini alamadım. Lütfen konumu kontrol edip tekrar deneyin."


In [ ]:
# Döviz kuru verilerini çekmek ve döviz dönüşümü yapmak için fonksiyon
def fetch_exchange_rate(base_currency, target_currency, amount=1):
    url = f"https://api.exchangeratesapi.io/v1/latest?access_key={exchange_api_key}&format=1"
    response = requests.get(url)
    data = response.json()
    rates = data.get('rates', {})
    if target_currency in rates:
        rate = rates[target_currency]
        converted_amount = float(amount) * rate
        return f"{amount} {base_currency} = {converted_amount:.2f} {target_currency}."
    else:
        return f"{target_currency} için döviz kuru bilgisi bulunamadı."


In [ ]:
def format_chat_history(chat_history):
    formatted_history = ""
    for entry in chat_history:
        if entry["role"] == "user":
            formatted_history += f"<div class='chat-bubble user'>{entry['content']}</div>"
        else:
            formatted_history += f"<div class='chat-bubble assistant'>{entry['content']}</div>"

    return formatted_history

In [ ]:
# GPT-4 Yanıtını oluşturmak için fonksiyon (function calling ile)
def generate_gpt4_response(prompt_input):
    global vector_index, documents, chat_history
    openai.api_key = openai_api_key

    functions = [
        {
            "name": "fetch_weather",
            "description": "Belirli bir konum için hava durumu bilgisini alır.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Hava durumu almak istediğiniz konumun adı"
                    }
                },
                "required": ["location"],
            },
        },
        {
            "name": "fetch_exchange_rate",
            "description": "Belirli iki para birimi arasındaki döviz kuru bilgisini alır ve isteğe bağlı olarak belirli bir miktar için dönüştürme yapar.",
            "parameters": {
                "type": "object",
                "properties": {
                    "base_currency": {
                        "type": "string",
                        "description": "Döviz kurunu almak istediğiniz temel para birimi, varsayılan olarak EUR'dir."
                    },
                    "target_currency": {
                        "type": "string",
                        "description": "Döviz kurunu almak istediğiniz hedef para biriminin ISO 4217 kodu (örneğin: TRY)"
                    },
                    "amount": {
                        "type": "number",
                        "description": "Dönüştürmek istediğiniz miktar (varsayılan olarak 1)."
                    }
                },
                "required": ["base_currency", "target_currency", "amount"],
            },
        }
    ]

    # OpenAI API çağrısı (function calling ile)
    response = openai.ChatCompletion.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system",
             "content": "Sen tıp bilgileri ile donatılmış bir asistansın ve görevin tıbbi konulardaki sorulara cevap vermektir."},
            {"role": "user", "content": prompt_input}
        ],
        functions=functions,
        function_call="auto",  # Modelin fonksiyon çağrısına karar vermesine izin ver
        temperature=0.5,
        max_tokens=512
    )

    # Sohbet geçmişini güncelle
    chat_history.append({"role": "user", "content": prompt_input})

    # Bir fonksiyon çağrısı istenip istenmediğini kontrol et
    if 'choices' in response and response['choices'][0]['finish_reason'] == 'function_call':
        function_call_info = response['choices'][0]['message']['function_call']
        function_name = function_call_info['name']
        arguments = json.loads(function_call_info['arguments'])

        if function_name == 'fetch_weather':
            location = arguments['location']
            weather_response = fetch_weather(location)
            chat_history.append({"role": "assistant", "content": weather_response})
            return format_chat_history(chat_history)
        elif function_name == 'fetch_exchange_rate':
            base_currency = arguments['base_currency']
            target_currency = arguments['target_currency']
            amount = arguments.get('amount', 1)
            exchange_rate_response = fetch_exchange_rate(base_currency, target_currency, amount)
            chat_history.append({"role": "assistant", "content": exchange_rate_response})
            return format_chat_history(chat_history)

    # Asistanın yanıtını al
    assistant_response = response['choices'][0]['message']['content'].strip()
    chat_history.append({"role": "assistant", "content": assistant_response})

    return format_chat_history(chat_history)

In [ ]:
with gr.Blocks() as demo:
    # CSS ile kaydırma özelliği ve stil düzenlemeleri
    custom_css = """
    /* Chat balonları için stil */
    .chat-bubble {
        padding: 10px;
        border-radius: 10px;
        margin-bottom: 10px;
        max-width: 60%;
        word-wrap: break-word;
    }

    .user {
        background-color: #d1e7dd;
        text-align: right;
        margin-left: auto;
    }

    .assistant {
        background-color: #f8d7da;
        text-align: left;
        margin-right: auto;
    }

    /* Kaydırılabilir sohbet kutusu */
    #output-box {
        height: 400px;  /* Sabit yükseklik */
        width: 100%;
        overflow-y: scroll !important;  /* Kaydırmayı gizle ama JS ile açacağız */
        padding: 10px;
        border: 1px solid #ccc;
        border-radius: 10px;
        background-color: #f8f9fa;
        margin-bottom: 20px;
    }

    /* Giriş kutusu */
    #input-box {
        height: 150px;
        width: 100%;
    }
    """

    output_textbox = gr.HTML(label="Yanıt", elem_id="output-box")  # HTML bileşeni ile sohbeti gösteriyoruz
    input_textbox = gr.Textbox(label="Sorunuzu girin", lines=4, elem_id="input-box")  # Giriş alanı

    # Sorgu gönderildiğinde çalıştırılacak fonksiyon
    def on_submit(prompt_input):
        response = generate_gpt4_response(prompt_input)
        return response, ""  # Giriş kutusunu temizle

    # Sohbeti temizlemek için fonksiyon
    def clear_chat():
        global chat_history
        chat_history.clear()
        return "", ""  # Hem giriş kutusunu hem de sohbeti temizle

    # Giriş kutusuna "Enter" basıldığında çalıştırılan fonksiyon
    input_textbox.submit(on_submit, inputs=input_textbox, outputs=[output_textbox, input_textbox])

    # Mesaj gönderme ve sohbeti temizleme butonları
    submit_btn = gr.Button("Gönder")
    clear_btn = gr.Button("Chat'i Temizle")

    submit_btn.click(on_submit, inputs=input_textbox, outputs=[output_textbox, input_textbox])
    clear_btn.click(clear_chat, outputs=[output_textbox, input_textbox])

    demo.css = custom_css
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1046f76164fb165e08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b162ad70725dccd76c.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://2eae138fb007aae19d.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://1dcffbd5ed4db3ce8d.gradio.live
Killing tunnel 127.0.0.1:7863 <> https://c9b054571b98902445.gradio.live
Killing tunnel 127.0.0.1:7864 <> https://6311d0f05878886aba.gradio.live
Killing tunnel 127.0.0.1:7865 <> https://44bc8391d8b84b0e1d.gradio.live
Killing tunnel 127.0.0.1:7866 <> https://1046f76164fb165e08.gradio.live


In [1]:
!pip install gradio==3.50.2
!pip install transformers==4.37.0
!pip install torch==2.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninstall: pillow
    Found existing installation: pillow 11.2.1
    Uninstalling pillow-11.2.1:
      Suc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.8 MB/s eta 0:00:00
^C
^C


In [5]:
import gradio as gr
from transformers import pipeline
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
from PIL import Image
import logging
from typing import Optional, Tuple, List, Dict

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class MedicalImageAnalyzer:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.image_processor = None
        self.image_model = None
        self.nlp_processor = None
        self.load_models()

    def load_models(self):
        """Load both image and NLP models"""
        try:
            logger.info("Loading BLIP-2 model for image analysis...")
            self.image_processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
            self.image_model = Blip2ForConditionalGeneration.from_pretrained(
                "Salesforce/blip2-opt-2.7b",
                torch_dtype=torch.float16,
                device_map="auto"
            )

            logger.info("Loading NLP models...")
            self.nlp_processor = pipeline(
                "text-generation",
                model="stanford-crfm/BioMedLM",
                device=self.device
            )

            logger.info("All models loaded successfully")
        except Exception as e:
            logger.error(f"Model loading failed: {e}")
            raise RuntimeError("Failed to initialize models. Please check logs.")

    def analyze_image(self, image: Image.Image, clinical_context: str = "") -> Dict:
        """Analyze medical image with optional clinical context"""
        try:
            if not image:
                raise ValueError("No image provided")

            # Generate radiology findings
            prompt = self._build_radiology_prompt(clinical_context)
            inputs = self.image_processor(image, text=prompt, return_tensors="pt").to(self.device, torch.float16)

            with torch.no_grad():
                outputs = self.image_model.generate(**inputs, max_new_tokens=300)

            findings = self.image_processor.decode(outputs[0], skip_special_tokens=True)
            findings = findings.replace(prompt, "").strip()

            # Generate differential diagnosis
            dd_prompt = f"As a radiologist, suggest differential diagnoses for these findings:\n{findings}\n\nDifferential Diagnoses:"
            dd_response = self.nlp_processor(
                dd_prompt,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True
            )
            differentials = dd_response[0]["generated_text"].replace(dd_prompt, "").strip()

            return {
                "findings": findings,
                "differentials": differentials,
                "recommendations": self._generate_recommendations(findings)
            }

        except Exception as e:
            logger.error(f"Analysis error: {e}")
            return {"error": str(e)}

    def _build_radiology_prompt(self, clinical_context: str) -> str:
        """Build structured radiology prompt"""
        return (
            "As a board-certified radiologist, carefully analyze this medical image. "
            f"Clinical context: {clinical_context if clinical_context else 'Not provided'}. "
            "Provide a professional interpretation noting:\n"
            "1. Anatomical structures visualized\n"
            "2. Abnormal findings\n"
            "3. Technical quality assessment\n"
            "4. Comparison to prior if available\n\n"
            "Radiological Interpretation:"
        )

    def _generate_recommendations(self, findings: str) -> str:
        """Generate clinical recommendations based on findings"""
        prompt = (
            "As a radiologist, provide clinical recommendations based on these findings:\n"
            f"{findings}\n\n"
            "Recommendations:"
        )
        response = self.nlp_processor(
            prompt,
            max_new_tokens=200,
            temperature=0.5,
            do_sample=True
        )
        return response[0]["generated_text"].replace(prompt, "").strip()

# Initialize the analyzer
analyzer = MedicalImageAnalyzer()

def analyze_medical_image(image: Image.Image, clinical_context: str = "") -> Dict:
    """Wrapper function for Gradio interface"""
    try:
        if not image:
            return "⚠️ Please upload a medical image"

        results = analyzer.analyze_image(image, clinical_context)

        if "error" in results:
            return f"❌ Error: {results['error']}"

        return (
            f"**Clinical Context**: {clinical_context if clinical_context else 'None provided'}\n\n"
            f"**Radiological Findings**:\n{results['findings']}\n\n"
            f"**Differential Diagnoses**:\n{results['differentials']}\n\n"
            f"**Recommendations**:\n{results['recommendations']}\n\n"
            "Note: This AI analysis should be verified by a qualified radiologist."
        )

    except Exception as e:
        logger.error(f"Processing error: {e}")
        return f"❌ Processing error: {str(e)}"

# Gradio Interface
with gr.Blocks(theme=gr.themes.Soft(), title="Medical Image Analysis") as app:
    gr.Markdown("""
    # 🩺 Advanced Medical Image Analysis
    *Upload diagnostic images for comprehensive radiological analysis*
    """)

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(
                type="pil",
                label="Upload Medical Image",
                sources=["upload", "clipboard"]
            )
            context_input = gr.Textbox(
                label="Clinical Context (optional)",
                placeholder="e.g., 45yo male with chronic leg pain...",
                lines=3
            )
            analyze_btn = gr.Button("Analyze", variant="primary")

        with gr.Column():
            report_output = gr.Markdown(
                label="Radiology Report",
                value="Analysis report will appear here..."
            )

    analyze_btn.click(
        analyze_medical_image,
        inputs=[image_input, context_input],
        outputs=report_output
    )

if __name__ == "__main__":
    try:
        app.launch(server_name="0.0.0.0", server_port=13)
    except Exception as e:
        logger.error(f"Application failed: {e}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
<ipython-input-5-7db77c085ecf>:146: GradioUnusedKwargWarning: You have unused kwarg parameters in Image, please remove them: {'sources': ['upload', 'clipboard']}
  image_input = gr.Image(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://9d34668a67e4df87c9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# This is the first model implementation and code implementation